In [17]:
import pandas as pd
import numpy as np
import math
import matplotlib.pyplot as plt
import datetime

from tqdm import tqdm
from dateutil.relativedelta import relativedelta
from pandas.tseries.offsets import DateOffset

In [18]:
ltg_df = pd.read_csv('data/LTG_projections.csv', parse_dates=['STATPERS'])

In [19]:
ltg_df['STATPERS_month'] = ltg_df['STATPERS'].dt.month
ltg_df['STATPERS_year'] = ltg_df['STATPERS'].dt.year
ltg_df['STATPERS_month_year'] = pd.to_datetime(ltg_df['STATPERS']).dt.to_period('M')

In [20]:
ltg_df = ltg_df.drop_duplicates(['TICKER', 'STATPERS_month_year']).drop(columns=['STATPERS_month_year'])

In [21]:
ltg_subset_df = ltg_df[['TICKER', 'STATPERS', 'MEDEST', 'MEANEST']]
ltg_subset_df['STATPERS_match_p1'] = ltg_subset_df['STATPERS'] - pd.DateOffset(months=1)
ltg_subset_df['STATPERS_match_month_p1'] = ltg_subset_df['STATPERS_match_p1'].dt.month
ltg_subset_df['STATPERS_match_year_p1'] = ltg_subset_df['STATPERS_match_p1'].dt.year

ltg_subset_df['STATPERS_match_p2'] = ltg_subset_df['STATPERS'] - pd.DateOffset(months=2)
ltg_subset_df['STATPERS_match_month_p2'] = ltg_subset_df['STATPERS_match_p2'].dt.month
ltg_subset_df['STATPERS_match_year_p2'] = ltg_subset_df['STATPERS_match_p2'].dt.year

ltg_subset_df['STATPERS_match_p3'] = ltg_subset_df['STATPERS'] - pd.DateOffset(months=3)
ltg_subset_df['STATPERS_match_month_p3'] = ltg_subset_df['STATPERS_match_p3'].dt.month
ltg_subset_df['STATPERS_match_year_p3'] = ltg_subset_df['STATPERS_match_p3'].dt.year

ltg_subset_df['STATPERS_match_l1'] = ltg_subset_df['STATPERS'] + pd.DateOffset(months=1)
ltg_subset_df['STATPERS_match_month_l1'] = ltg_subset_df['STATPERS_match_l1'].dt.month
ltg_subset_df['STATPERS_match_year_l1'] = ltg_subset_df['STATPERS_match_l1'].dt.year

ltg_subset_df = ltg_subset_df.drop(columns=['STATPERS'])
matching_columns = ['STATPERS_match_p1', 'STATPERS_match_month_p1', 'STATPERS_match_year_p1', 
                    'STATPERS_match_p2', 'STATPERS_match_month_p2', 'STATPERS_match_year_p2', 
                    'STATPERS_match_l1', 'STATPERS_match_month_l1', 'STATPERS_match_year_l1']

<ipython-input-21-f09869e358ec>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ltg_subset_df['STATPERS_match_p1'] = ltg_subset_df['STATPERS'] - pd.DateOffset(months=1)
<ipython-input-21-f09869e358ec>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ltg_subset_df['STATPERS_match_month_p1'] = ltg_subset_df['STATPERS_match_p1'].dt.month
<ipython-input-21-f09869e358ec>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value

In [22]:
ltg_df = pd.merge(ltg_df, 
                  ltg_subset_df, 
                  left_on=['TICKER', 'STATPERS_month', 'STATPERS_year'], 
                  right_on=['TICKER', 'STATPERS_match_month_p1', 'STATPERS_match_year_p1'], 
                  suffixes=[None, '_p1'], 
                  how='left').drop(columns=matching_columns)

ltg_df = pd.merge(ltg_df, 
                  ltg_subset_df, 
                  left_on=['TICKER', 'STATPERS_month', 'STATPERS_year'], 
                  right_on=['TICKER', 'STATPERS_match_month_p2', 'STATPERS_match_year_p2'], 
                  suffixes=[None, '_p2'], 
                  how='left').drop(columns=matching_columns)

ltg_df = pd.merge(ltg_df, 
                  ltg_subset_df, 
                  left_on=['TICKER', 'STATPERS_month', 'STATPERS_year'], 
                  right_on=['TICKER', 'STATPERS_match_month_p3', 'STATPERS_match_year_p3'], 
                  suffixes=[None, '_p3'], 
                  how='left').drop(columns=matching_columns)

ltg_df = pd.merge(ltg_df, 
                  ltg_subset_df, 
                  left_on=['TICKER', 'STATPERS_month', 'STATPERS_year'], 
                  right_on=['TICKER', 'STATPERS_match_month_l1', 'STATPERS_match_year_l1'], 
                  suffixes=[None, '_l1'], 
                  how='left').drop(columns=matching_columns)

In [23]:
ltg_df['chg_MEDEST_2'] = ltg_df['MEDEST_p1'] - ltg_df['MEDEST_l1']
ltg_df['chg_MEANEST_2'] = ltg_df['MEANEST_p1'] - ltg_df['MEANEST_l1']
ltg_df['chg_MEDEST_3'] = ltg_df['MEDEST_p2'] - ltg_df['MEDEST_l1']
ltg_df['chg_MEANEST_3'] = ltg_df['MEANEST_p2'] - ltg_df['MEANEST_l1']
ltg_df['chg_MEANEST_rev21'] = ltg_df['MEANEST_p2'] - ltg_df['MEANEST_p1']
ltg_df['chg_MEDEST_rev21'] = ltg_df['MEDEST_p2'] - ltg_df['MEDEST_p1']
ltg_df['chg_MEANEST_rev32'] = ltg_df['MEANEST_p3'] - ltg_df['MEANEST_p2']
ltg_df['chg_MEDEST_rev32'] = ltg_df['MEDEST_p3'] - ltg_df['MEDEST_p2']
ltg_df['chg_MEANEST_rev31'] = ltg_df['MEANEST_p3'] - ltg_df['MEANEST_p1']
ltg_df['chg_MEDEST_rev31'] = ltg_df['MEDEST_p3'] - ltg_df['MEDEST_p1']

In [24]:
ltg_df.to_pickle('data/checkpoint_data/LTG.pkl')